# Propositional logic with exhaustification (ExhPL)

In [24]:
import Prelude hiding (lookup)

## Syntax

We start by extending the datatype for expressions of Propositional Logic from the `hatt` library.

### Variables

Variables are simply characters. Variables with the same name will always be assigned the same truth-value by a given assignment.

In [18]:
newtype Var = Var Char
    deriving (Eq, Ord)

instance Show Var where
    show (Var v) = [v]

### Wffs

Following the `hatt` library, we define a recursive datatype for wffs of exhaustified propositional logic. The only addition here is the `Exh` data constructor.

In [18]:
data Expr = Variable Var
          | Negation Expr
          | Exh Expr
          | Conjunction Expr Expr
          | Disjunction Expr Expr
          deriving Eq

Here, we define some functions to display wffs of ExhPL in a more readable fashion.

In [18]:
    
showBinaryConnective :: (Expr -> String) -> String -> Expr -> Expr -> String
showBinaryConnective show_ symbol exp1 exp2 =
  '(' : show_ exp1 ++ " " ++ symbol ++ " " ++ show_ exp2 ++ ")"

showBC :: String -> Expr -> Expr -> String
showBC = showBinaryConnective show
          
instance Show Expr where
  show (Variable      name)      = show name
  show (Negation      expr)      = '¬' : show expr
  show (Exh expr) = 'O' : show expr
  show (Conjunction   exp1 exp2) = showBC "∧" exp1 exp2
  show (Disjunction   exp1 exp2) = showBC "∨" exp1 exp2

## TODO Parsing expressions with `parsec`

### Computing Alternatives

In [56]:
binaryConnectives :: [(Expr -> Expr -> Expr)]
binaryConnectives = [Conjunction,Disjunction]

class Alt a where
    alt :: a -> [a]

instance Alt Expr where
    alt (Negation exp) = (alt exp) ++ map Negation (alt exp)
    alt (Conjunction exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- (alt exp1), exp2' <- (alt exp2)] ++ (alt exp1) ++ (alt exp2)
    alt (Disjunction exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- (alt exp1), exp2' <- (alt exp2)] ++ (alt exp1) ++ (alt exp2)
    alt exp = [exp]

In [57]:
alt $ Variable (Var 'a') `Disjunction` Variable (Var 'b') `Disjunction` Variable (Var 'c')

[((a ∧ b) ∧ c),((a ∨ b) ∧ c),(a ∧ c),(b ∧ c),((a ∧ b) ∨ c),((a ∨ b) ∨ c),(a ∨ c),(b ∨ c),(a ∧ b),(a ∨ b),a,b,c]

## Semantics

In [83]:
import Data.Map (Map, fromList, lookup)
import Control.Monad (liftM, liftM2, replicateM)
import Data.Maybe (fromMaybe)
import Data.List (group, sort)

Formulae are interpreted relative to assignments of variables to truth-values.

In [61]:
type Mapping = Map Var Bool

interpret :: Expr -> Mapping -> Bool
interpret (Variable      v)         vs = fromMaybe False (lookup v vs)
interpret (Negation      expr)      vs = not $ interpret expr vs
interpret (Exh expr) vs = interpret (exh expr) vs
interpret (Conjunction   exp1 exp2) vs = interpret exp1 vs && interpret exp2 vs
interpret (Disjunction   exp1 exp2) vs = interpret exp1 vs || interpret exp2 vs

assignments :: Expr -> [Mapping]
assignments expr = let vs = variables expr
                       ps = replicateM (length vs) [True, False]
                   in  map (fromList . zip vs) ps
                   
variables :: Expr -> [Var]
variables expr = let vars_ (Variable      v)     vs = v : vs
                     vars_ (Negation      e)     vs = vars_ e vs
                     vars_ (Exh e) vs = vars_ e vs
                     vars_ (Conjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (Disjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                 in  map head . group . sort $ vars_ expr []
                 
values :: Expr -> [Bool]
values expr = map (interpret expr) (assignments expr)

-- | Determines whether an expression is contradictory.
isContradiction :: Expr -> Bool
isContradiction = not . or . values

Given a prejacent `prej`, and an alternative `alt`, `excl` determines whether or not `alt` is *excludable* relative to `prej`. Here, we assume that an alternative is excludable if it is non-truth-conditionally weaker.

In [62]:
-- | Determines whether or not an alternative is excludable
excl :: Expr -> Expr -> Bool
excl prej alt = (not . isContradiction) (Conjunction prej (Negation alt))

Now, we define a function `exclAlts`, which for a given expression `expr`, computes the set of excludable alternatives to `expr`.

In [63]:
-- | Returns the list of excludable alternatives
exclAlts :: Expr -> [Expr]
exclAlts expr = filter (excl expr) (alt expr)

Finally, we define a function `exh` that takes an expression, and returns the (syntactic) conjunction of the expression with the negation of each of its excludable alternatives. We could do this directly in the semantics, but it will be useful to see the result of applying `Exh`.

In [64]:
exh :: Expr -> Expr
exh exp = foldr Conjunction exp (map Negation $ exclAlts exp)

In [68]:
values $ exh (Variable (Var 'a') `Disjunction` (Variable (Var 'b')))

Line 1: Redundant bracket
Found:
Variable (Var 'a') `Disjunction` (Variable (Var 'b'))
Why not:
Variable (Var 'a') `Disjunction` Variable (Var 'b')

[False,False,False,False]

## Innocent Exclusion

First, we need a function for computing the subsets.

In [69]:
subsets :: [a] -> [[a]]
subsets []  = [[]]
subsets (x:xs) = subsets xs ++ map (x:) (subsets xs)

In [94]:
subsets $ exclAlts $ Variable (Var 'a') `Disjunction` Variable (Var 'b')

[[],[b],[a],[a,b],[(a ∧ b)],[(a ∧ b),b],[(a ∧ b),a],[(a ∧ b),a,b]]

In [72]:
isConsistent :: Expr -> [Expr] -> Bool
isConsistent expr alts = (not . isContradiction) $ foldr Conjunction expr (map Negation alts)

In [90]:
import Data.Function (on)
import Data.List (maximumBy)

on :: forall b c a. (b -> b -> c) -> (a -> b) -> a -> a -> c

In [91]:
longest = maximumBy (compare `on` length)

In [92]:
longest $ subsets [1,2,3]

[1,2,3]